In [1]:
exec(open("./funcs/tool_funcs.py").read())



In [2]:
# For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
 

def dijkstra_one_to_all_widest_path(distance_matrix, src, need_up_i, remove_node):
    
    N = len(distance_matrix)
    
#     import pdb;pdb.set_trace()
    
    dist = [0] * N
    dist[src] = np.inf
    visited = [False] * N
 
    # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
    pq = [(-np.inf, src)]
    
    kk = len(need_up_i)
    counter = 0
    visited[remove_node] = True

    while pq:
        current_dist, u = heapq.heappop(pq)
        # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
        current_dist = -current_dist

        if visited[u]:
            continue
        visited[u] = True
        
        if u in need_up_i:
            counter += 1
            if counter == kk:
                
                return np.array(dist)
 

        # Update distances for neighbors
        for v in range(N):
            if not visited[v] and distance_matrix[u][v] < np.inf:
                new_dist = min(current_dist, distance_matrix[u][v])
                if new_dist > dist[v]:
                    dist[v] = new_dist
                    # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
                    heapq.heappush(pq, (-new_dist, v))

    return np.array(dist)



In [3]:
 def cal_need_update_list_widest_path(remove_node, X_widest_path_matrix):
    N = len(X_widest_path_matrix)
    removal = remove_node
    remaining_list = [i for i in np.arange(N) if i != removal]
    need_update_list = [[] for i in range(N)]
    how_many_counter = np.zeros(N)
    for i in remaining_list:
        for j in remaining_list:
            if i != j:
                if X_widest_path_matrix[i,j] <= min(X_widest_path_matrix[i,removal], X_widest_path_matrix[removal,j]):
                    need_update_list[i].append(j)
                    how_many_counter[i] += 1
    
    how_many = np.sum(how_many_counter > 0)
    
    
    return need_update_list, how_many


In [4]:
def generate_new_distance_matrix_widest_path(X_distance_matrix, remove_node):
    X_distance_matrix_new = X_distance_matrix.copy()
 

    N = len(X_distance_matrix)
    for i in range(N):
        for j in range(N):
            if i == remove_node or j == remove_node:
                X_distance_matrix_new[i,j] = 0

    X_distance_matrix_new[remove_node,remove_node] = np.inf  
    
    return X_distance_matrix_new

In [5]:

def cal_new_widest_path_matrix_warm(X_distance_matrix, X_widest_path_matrix, remove_node):
    N = len(X_distance_matrix)
    X_widest_path_matrix_new = X_widest_path_matrix.copy()
    remaining_list = [i for i in np.arange(N) if i != remove_node]

    need_update_list, how_many = cal_need_update_list_widest_path(remove_node, X_widest_path_matrix)
    
    Cost = np.round(how_many/(N - 1), 3)
    
    print("Cost: ", Cost)

    X_distance_matrix_new = generate_new_distance_matrix_widest_path(X_distance_matrix, remove_node)
 
    for i in remaining_list:       
        if len(need_update_list[i]) > 0:
            temp = dijkstra_one_to_all_widest_path(X_distance_matrix_new, i, need_update_list[i], remove_node)

            for j in need_update_list[i]:
                X_widest_path_matrix_new[i,j] = temp[j]
 
            
 
    for i in range(N):
        for j in range(N):
            if i == remove_node or j == remove_node:
                X_widest_path_matrix_new[i,j] = 0

    X_widest_path_matrix_new[remove_node,remove_node] = np.inf
    
    return X_widest_path_matrix_new 


In [6]:
def widest_path_n_to_r_new(distance_matrix, widest_path_matrix, remaining_list, r, remove_node):
  
    max_jump_list = []
    for t in remaining_list:
        m_jump =  min(distance_matrix[remove_node,t], widest_path_matrix[t,r])
        max_jump_list.append(m_jump)
    return np.max(max_jump_list)

def widest_path_r_to_n_new(distance_matrix, widest_path_matrix, remaining_list, r, remove_node):
    max_jump_list = []
    for t in remaining_list:
        m_jump =  min(widest_path_matrix[r,t], distance_matrix[t, remove_node]) 
        max_jump_list.append(m_jump)
    return np.max(max_jump_list)

     
def cal_n_widest_path_new(distance_matrix, widest_path_matrix, remove_node, remaining_list):
    for r in remaining_list:
        widest_path_matrix[remove_node, r] = widest_path_n_to_r_new(distance_matrix, widest_path_matrix, remaining_list, r, remove_node)
        widest_path_matrix[r, remove_node] = widest_path_r_to_n_new(distance_matrix, widest_path_matrix, remaining_list, r, remove_node)
        
    for i in remaining_list:        
        for j in remaining_list:
            if i < j:
                widest_path_matrix[i,j] =  update_widest_path_ij_new(distance_matrix, widest_path_matrix, remove_node, i, j)
                widest_path_matrix[j,i] =  update_widest_path_ij_new(distance_matrix, widest_path_matrix, remove_node, j, i)
                
def update_widest_path_ij_new(distance_matrix, widest_path_matrix, remove_node, i,j):
    m1 = widest_path_matrix[i,j]
    m2 =  min(widest_path_matrix[i,remove_node], widest_path_matrix[remove_node,j])
    return np.max((m1,m2))



In [7]:
def cal_which_to_remove(X_widest_path_matrix, edge_nodes):
   
    _, how_many0 = cal_need_update_list_widest_path(edge_nodes[0], X_widest_path_matrix)
    _, how_many1 = cal_need_update_list_widest_path(edge_nodes[1], X_widest_path_matrix)
    if how_many0 < how_many1:
        remove_node = edge_nodes[0]
    else:
        remove_node = edge_nodes[1]
    # print(how_many0, how_many1, edge_nodes, remove_node)
    return remove_node
    
    
    

In [8]:

def cal_widest_path_matrix_after_graph_update(X_distance_matrix, X_widest_path_matrix, edge_nodes, X_distance_matrix_updated):
    
    N = len(X_distance_matrix)
    
    remove_node = cal_which_to_remove(X_widest_path_matrix, edge_nodes)

    X_widest_path_matrix_updated = cal_new_widest_path_matrix_warm(X_distance_matrix, X_widest_path_matrix, remove_node)
 
    remaining_list = [i for i in np.arange(N) if i != remove_node]
    
    cal_n_widest_path_new(X_distance_matrix_updated, X_widest_path_matrix_updated, remove_node, remaining_list)
    
    return X_widest_path_matrix_updated

 
    

In [9]:
# N = 100

# X_distance_matrix = create_distance_matrix(N)

In [10]:
X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")
 

In [11]:

# For the widest path problem, a node's bandwidth to itself should be infinity.


N = len(X_distance_matrix)
for i in range(N):
    X_distance_matrix[i,i] = np.inf

In [12]:

edge_nodes = [3, 9]

edge_weight = 50    

X_distance_matrix_new = X_distance_matrix.copy()
X_distance_matrix_new[edge_nodes[0],edge_nodes[1]] = edge_weight 

X_widest_path_matrix = floyd_warshall_widest_path(X_distance_matrix)


In [13]:

start = time.time()
X_widest_path_matrix_new_warm = cal_widest_path_matrix_after_graph_update(X_distance_matrix, X_widest_path_matrix, edge_nodes, X_distance_matrix_new)
end = time.time()


time_used1_warm = end - start
time_used1_warm = np.round(time_used1_warm, 3)

Cost:  1.0


In [14]:
print(f"Time used: {time_used1_warm}s" )


Time used: 0.492s


In [15]:
start = time.time()
X_widest_path_matrix_new_floyd_warshall = floyd_warshall_widest_path(X_distance_matrix_new)
end = time.time()


time_used2_floyd = end - start
time_used2_floyd = np.round(time_used2_floyd, 3)


In [16]:
print(f"Time used: {time_used2_floyd}s" )


Time used: 1.153s


In [17]:
print(np.allclose(X_widest_path_matrix_new_warm, X_widest_path_matrix_new_floyd_warshall))


True


In [18]:
print(f"Ratio: {np.round(time_used1_warm/time_used2_floyd, 2)}")


Ratio: 0.43


In [19]:
X_widest_path_matrix_new_warm[1]


array([ 984.,   inf,  984.,  960.,  982.,  984.,  984.,  984.,  982.,
        984.,  975.,  984.,  984.,  983.,  950.,  984.,  984.,  984.,
        984.,  984.,  981.,  984.,  984.,  976.,  984.,  984.,  984.,
        984.,  977.,  984.,  960.,  982.,  984.,  983., 1000.,  984.,
        984.,  984.,  976.,  981.,  982.,  984.,  984.,  984.,  970.,
        984.,  972.,  984.,  984.,  984.,  984.,  984.,  984.,  976.,
        980.,  984.,  984.,  984.,  973.,  984.,  984.,  984.,  984.,
        984.,  984.,  984.,  984.,  975.,  983.,  984.,  980.,  984.,
        984.,  976.,  984.,  984.,  984.,  984.,  984.,  981.,  984.,
        984.,  980.,  984.,  982.,  984.,  965.,  984.,  984.,  984.,
        975.,  984.,  984.,  983.,  984.,  984.,  984.,  984.,  984.,
        984.])